In [2]:
using CSV;
using DataFrames;
using DelimitedFiles;
using LinearAlgebra;
using Plots

In [3]:
pwd()

"/home/zyu20/git/BulkLMM.jl/analysis/HSNIH-Palmer"

In [5]:
results_BulkLMM = CSV.read("output/BulkLMM_results.csv", DataFrame; header = false) |> Matrix;

In [11]:
function p2lod(pval::Float64)
    
    lrs = -log10(pval)*2*log(10)
    lod = lrs/(2*log(10))
    
    # return lrs
    return lod

end

p2lod (generic function with 1 method)

In [12]:
lods_gemma = map(x -> p2lod(x), results_GEMMA[2:end, 10]);

In [13]:
lods_BulkLMM = results_BulkLMM[3:end, 1];

In [20]:
length(lods_BulkLMM)

102279

In [14]:
summary = hcat(lods_gemma, lods_BulkLMM);
summary[1:6, :]

6×2 Matrix{Float64}:
 2.5677    1.95344
 0.468855  0.197911
 2.60785   1.99015
 1.94852   1.39496
 2.13121   1.55804
 1.98993   1.43178

In [30]:
tol = 0.5

0.5

In [31]:
sum((lods_gemma .- lods_BulkLMM) .<= tol)/102279

0.9700427262683444

In [15]:
thr = 3.0

3.0

In [16]:
lods_BulkLMM[lods_BulkLMM .> thr]

6-element Vector{Float64}:
 3.076760458662261
 3.1808702870693564
 3.212289714830687
 3.2808192698589966
 3.011824418149289
 3.1575146035897332

In [17]:
sig_BulkLMM = findall(x -> (x>thr), lods_BulkLMM)

6-element Vector{Int64}:
 83887
 83893
 83904
 83915
 83918
 83922

In [17]:
sig_gemma = findall(x -> (x>thr), lods_gemma)

47-element Vector{Int64}:
  2269
 20089
 20096
 20118
 20122
 20178
 20184
 20187
 20188
 23100
 23118
 48823
 76536
     ⋮
 84263
 84266
 84269
 84275
 84468
 84476
 85249
 85252
 85259
 85313
 89852
 89855

In [18]:
map(x -> x in sig_gemma, sig_BulkLMM) 

6-element Vector{Bool}:
 1
 1
 1
 1
 1
 1